### NEXT STEPS:



- Quitar los *outlires*
- Probar con los "sin estandardizar"
- Mirar la importancia de cada columna para quitarlas o menos

### Sprint 2 - Random Forest

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

/home/sanfermen/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("../datos/bikes_casual_estan.csv", index_col = 0)
df.head()

,instant,season,yr,mnth,holiday,workingday,weathersit,temp,atemp,hum,windspeed,casual,weekday
0,1,0,0,1,1,0,1,14.110847,-0.456399,0.855582,-0.208909,331,2
1,2,0,0,2,0,1,1,14.902598,-0.492863,0.332722,0.687550,131,0
2,3,0,0,3,0,1,2,8.050924,-1.098990,-0.900337,0.685210,120,0
3,4,1,0,4,0,0,2,8.200000,-1.015201,-0.170632,-0.210435,108,1
4,5,1,0,5,0,1,2,9.305237,-0.951952,-0.901843,0.060294,82,1


In [3]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X = df.drop("casual", axis = 1)
y = df["casual"]

In [4]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [5]:
# recordamos que lo primero que hacíamos era crear un diccionario con los hiperparámetros que queríamos probar

param = {"max_depth": [22], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split":[8],
        "min_samples_leaf": [8]} 

In [6]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [7]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [22], 'max_features': [4],
                         'min_samples_leaf': [8], 'min_samples_split': [8]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [8]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=22, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [9]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [10]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [11]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,303.956072,181727.783537,426.295418,0.621735,test,Random Forest
1,290.797050,163818.457395,404.744929,0.649741,train,Random Forest


In [32]:
df_r = pd.read_csv("../datos/bikes_registered_estan.csv", index_col = 0)
df_r.head()

,instant,season,yr,mnth,holiday,workingday,weathersit,temp,atemp,hum,windspeed,registered,weekday
0,1,0,0,1,1,0,1,14.110847,-0.456399,0.855582,-0.208909,654,0
1,2,0,0,2,0,1,1,14.902598,-0.492863,0.332722,0.687550,670,2
2,3,0,0,3,0,1,2,8.050924,-1.098990,-0.900337,0.685210,1229,2
3,4,1,0,4,0,0,2,8.200000,-1.015201,-0.170632,-0.210435,1454,0
4,5,1,0,5,0,1,2,9.305237,-0.951952,-0.901843,0.060294,1518,0


In [33]:
#df_r.drop("weekday", axis= 1, inplace= True)
#df_r

In [34]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X2 = df_r.drop("registered", axis = 1)
y2 = df_r["registered"]

In [35]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x2_train, x2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state = 42)

In [36]:
# recordamos que lo primero que hacíamos era crear un diccionario con los hiperparámetros que queríamos probar

param2 = {"max_depth": [16], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split":[8],
        "min_samples_leaf": [8]} 

In [37]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [38]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf2.fit(x2_train, y2_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [16], 'max_features': [4],
                         'min_samples_leaf': [8], 'min_samples_split': [8]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [39]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestRegressor(max_depth=16, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [40]:
y2_pred_test_rf = bosque2.predict(x2_test)
y2_pred_train_rf = bosque2.predict(x2_train)

In [41]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [42]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results2 = metricas(y2_test, y2_train, y2_pred_test_rf, y2_pred_train_rf, "Random Forest Registered I")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,506.492761,393557.347977,627.341492,0.817677,test,Random Forest Registered I
1,479.881772,402448.485160,634.388276,0.838574,train,Random Forest Registered I


Nos quedamos con *registered* porque tiene el R2 mejor.

- Quitar los put

In [43]:
param3 = {"max_depth": [16, 18], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split":[8, 20],
        "min_samples_leaf": [8, 20]} 

In [44]:
gs_rf3 = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param3, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [45]:
gs_rf3.fit(x2_train, y2_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [16, 18], 'max_features': [4],
                         'min_samples_leaf': [8, 20],
                         'min_samples_split': [8, 20]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [46]:
bosque3 = gs_rf3.best_estimator_
bosque3

RandomForestRegressor(max_depth=18, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [47]:
y_pred_test_rf3 = bosque3.predict(x2_test)
y_pred_train_rf3 = bosque3.predict(x2_train)

In [49]:

dt_results3 = metricas(y2_test, y2_train, y_pred_test_rf3, y_pred_train_rf3, "Random Forest Registered II")
dt_results3

,MAE,MSE,RMSE,R2,set,modelo
0,504.724284,392167.259900,626.232592,0.818321,test,Random Forest Registered II
1,478.862573,400018.186788,632.469910,0.839549,train,Random Forest Registered II


In [51]:
df_sinest_reg = pd.read_csv("../datos/bikes_registered_sinest.csv", index_col= 0)
df_sinest_reg.head()

,instant,season,yr,mnth,holiday,workingday,weathersit,temp,atemp,hum,windspeed,registered,weekday
0,1,0,0,1,1,0,1,14.110847,18.18125,80.5833,10.749882,654,0
1,2,0,0,2,0,1,1,14.902598,17.68695,69.6087,16.652113,670,2
2,3,0,0,3,0,1,2,8.050924,9.47025,43.7273,16.636703,1229,2
3,4,1,0,4,0,0,2,8.200000,10.60610,59.0435,10.739832,1454,0
4,5,1,0,5,0,1,2,9.305237,11.46350,43.6957,12.522300,1518,0


In [52]:
X3 = df_sinest_reg.drop("registered", axis = 1)
y3 = df_sinest_reg["registered"]

In [53]:
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.2, random_state = 42)

In [54]:
gs_rf_sines = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param3, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") 

In [55]:
gs_rf_sines.fit(x3_train, y3_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [16, 18], 'max_features': [4],
                         'min_samples_leaf': [8, 20],
                         'min_samples_split': [8, 20]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [56]:
bosque_sines = gs_rf_sines.best_estimator_
bosque_sines

RandomForestRegressor(max_depth=16, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [57]:
y_pred_test_rf_sines = bosque_sines.predict(x3_test)
y_pred_train_rf_sines = bosque_sines.predict(x3_train)

In [58]:
dt_results_sines = metricas(y3_test, y3_train, y_pred_test_rf_sines, y_pred_train_rf_sines, "Random Forest Registered Sinest I")
dt_results_sines

,MAE,MSE,RMSE,R2,set,modelo
0,502.993472,391290.518819,625.532188,0.818727,test,Random Forest Registered Sinest I
1,482.342134,410176.530606,640.450256,0.835474,train,Random Forest Registered Sinest I
